In [1]:
import numpy as np
import torch

from s3ts.helper_functions import *

In [2]:
dm = load_tsdataset("WISDM", "./datasets/", 128, 8, 48, 1, True, None, [30, 31, 32, 33, 34, 35], True)

Loaded dataset WISDM with a total of 1096480 observations for window size 48
Using 443636 observations for training and 154375 observations for validation and test


In [3]:
dm.stsds.SCS[dm.stsds.indices[dm.ds_train.indices]].unique(return_counts=True)

(tensor([0, 1, 2, 3, 4, 5]),
 tensor([88291, 88486, 50063, 40441, 88335, 88020]))

In [4]:
dm.stsds.SCS[dm.stsds.indices[dm.ds_test.indices]].unique(return_counts=True)

(tensor([0, 1, 2, 3, 4, 5]),
 tensor([12077, 38519,  9828,  7857, 14484, 71610]))

In [5]:
from s3ts.api.nets.methods import create_model_from_DM, train_model

model = create_model_from_DM(dm, "ts", "cnn", "mlp", "cls", "test", 20, 64, 2, 0.001)

Input shape:  torch.Size([1, 3, 48])
Latent shape:  torch.Size([1, 80, 12])
[tensor(960), 64, 64, 6]


In [6]:
train_model(dm, model, 5)

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

   | Name        | Type                 | Params
------------------------------------------------------
0  | encoder     | CNN_TS               | 12.9 K
1  | decoder     | MultiLayerPerceptron | 66.1 K
2  | flatten     | Flatten              | 0     
3  | softmax     | Softmax              | 0     
4  | train_acc   | MulticlassAccuracy   | 0     
5  | train_f1    | MulticlassF1Score    | 0     
6  | train_auroc | MulticlassAUROC      | 0     
7  | val_acc     | MulticlassAccuracy   | 0     
8  | val_f1      | MulticlassF1Score    | 0     
9  | val_auroc   | MulticlassAUROC      | 0     
10 | test_acc    | MulticlassAccuracy   | 0     
11 | test_f1     | MulticlassF1Score    | 0     
12 | test_auroc  | MulticlassAUROC      | 0     
------------------------------------------------------
79.0 K    Trainable params
0      

Epoch 4: 100%|██████████| 3466/3466 [01:04<00:00, 53.63it/s, v_num=3, train_loss_step=0.221, val_loss=0.665, val_acc=0.864, val_auroc=0.0125, train_loss_epoch=0.223, train_acc=0.922, train_auroc=0.993] 

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 3466/3466 [01:04<00:00, 53.61it/s, v_num=3, train_loss_step=0.221, val_loss=0.665, val_acc=0.864, val_auroc=0.0125, train_loss_epoch=0.223, train_acc=0.922, train_auroc=0.993]
Input shape:  torch.Size([1, 3, 48])
Latent shape:  torch.Size([1, 80, 12])
[tensor(960), 64, 64, 6]
Validation DataLoader 0: 100%|██████████| 1207/1207 [00:09<00:00, 128.71it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.8639352321624756     │
│         val_auroc         │   0.012527707032859325    │
│          val_f1           │    0.8639352321624756     │
│         val_loss          │    0.6652042865753174     │
└───────────────────────────┴───────────────────────────┘

(WrapperModel(
   (encoder): CNN_TS(
     (cnn_0): Sequential(
       (0): Conv1d(3, 10, kernel_size=(3,), stride=(1,), padding=same)
       (1): ReLU()
       (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     )
     (cnn_1): Sequential(
       (0): Conv1d(10, 20, kernel_size=(3,), stride=(1,), padding=same)
       (1): ReLU()
       (2): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
       (3): Dropout(p=0.35, inplace=False)
     )
     (cnn_2): Sequential(
       (0): Conv1d(20, 40, kernel_size=(3,), stride=(1,), padding=same)
       (1): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU()
       (3): Dropout(p=0.4, inplace=False)
     )
     (cnn_3): Conv1d(40, 80, kernel_size=(3,), stride=(1,), padding=same)
   )
   (decoder): MultiLayerPerceptron(
     (fcn_layer_0): Linear(in_features=960, out_features=64, bias=True)
     (act_layer_0): ReLU()
     (fcn_layer_1): Linear(in_features=64, out_

In [7]:
model.eval()

out = torch.zeros((len(dm.ds_test), 6))
lb = torch.zeros(len(dm.ds_test))
for i in range(len(dm.ds_test)):
    d = dm.ds_test[i]
    out[i, :] = model.logits(d["series"].unsqueeze(0))[0]
    lb[i] = d["label"]

In [8]:
import torchmetrics as tm

In [9]:
confm = tm.ConfusionMatrix(task="multiclass", num_classes=6)

In [10]:
acc = tm.Accuracy(task="multiclass", num_classes=6, average="macro")

In [11]:
p = tm.Precision(task="multiclass", num_classes=6, average="macro")
r = tm.Recall(task="multiclass", num_classes=6, average="macro")

In [12]:
auroc = tm.AUROC(task="multiclass", num_classes=6, average="macro")

In [13]:
lb.unique(return_counts=True)

(tensor([0., 1., 2., 3., 4., 5.]),
 tensor([12077, 38521,  9826,  7854, 14484, 71613]))

In [14]:
confm(out, lb)

tensor([[ 8765,   140,    44,    40,  2531,   557],
        [  870, 36989,     0,     0,   504,   158],
        [   36,     0,  8700,    78,  1012,     0],
        [   17,     0,    21,  7796,    20,     0],
        [ 1836,    18,    57,    81, 10766,  1726],
        [ 6397,   201,   266,     0,  4395, 60354]])

In [15]:
auroc(out, lb.long())

tensor(0.9608)

In [16]:
acc(out, lb.long())

tensor(0.8583)

In [17]:
p(out, lb.long())

tensor(0.8222)

In [18]:
r(out, lb.long())

tensor(0.8583)